In [ ]:
import numpy as np
import pandas as pd
import sys, os                               #sys用来获取命令行参数，os用来配置文件地址一些
from random import shuffle
import torch
import torch.nn as nn
from models.gat import GATNet               #models是文件夹名字，gat是模型py文件，这里意思是中这个文件里导入GATNet类
from models.gat_gcn import GAT_GCN
from models.gcn import GCNNet
from models.ginconv import GINConvNet
from utils import *


def train(model, device, train_loader, optimizer, epoch):                             #定义训练函数，参数包括模型，设备，训练集，优化器，迭代次数
    print('Training on {} samples...'.format(len(train_loader.dataset)))              #打印训练集样本数，用.format()函数
    model.train()                                                                     #首先将模型设置为训练模式
    for batch_idx, data in enumerate(train_loader):                                   #用enmerate()函数得到每个批次的数据集和索引，再遍历，这里train_loader不理解可以看下边解释                      
        data = data.to(device)                                                        #对于每个批次数据集加载到指定设备，下边也有定义，这里指的是CPU/GPU设备
        optimizer.zero_grad()                                                         #将优化器的梯度清零
        output = model(data)                                                          #通过模型计算出这一批次数据集输出
        loss = loss_fn(output, data.y.view(-1, 1).float().to(device))                 #首先将data的y值转化为二维浮点型张量并且加载到指定设备中，再通过损失函数计算其损失值，用于计算梯度，更新参数，模型评估，损失函数loss_fu后边有定义
        loss.backward()                                                               #将损失反向传播
        optimizer.step()                                                              #更新模型参数
        if batch_idx % LOG_INTERVAL == 0:                                             #如果这里定义个参数LOG每LOG批次打印以下如下包括迭代次数、遍历到的样本总数、样本总数、进行的百分比、损失值
            print('Train epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch,
                                                                           batch_idx * len(data.x),
                                                                           len(train_loader.dataset),
                                                                           100. * batch_idx / len(train_loader),
                                                                           loss.item()))

def predicting(model, device, loader):                                                #定于预测函数，参数有模型，设备，数据
    model.eval()                                                                      #由于是预测过程，将模型设为评估状态，此时只需要前向传播，不需要反向传播更新参数，一些dropout也不改变
    total_preds = torch.Tensor()                                                      #创建张量数据集，定义为预测变量
    total_labels = torch.Tensor()
    print('Make prediction for {} samples...'.format(len(loader.dataset)))            #打印预测样本数
    with torch.no_grad():                                                             #预测中禁用梯度计算
        for data in loader:                                                           #数据在测试数据集中迭代
            data = data.to(device)                                                    #数据导入设备
            output = model(data)                                                      #模型对数据进行预测得到输出
            total_preds = torch.cat((total_preds, output.cpu()), 0)                   #将输出数据集以二维张量形式按照样本数进行拼接得到预测值
            total_labels = torch.cat((total_labels, data.y.view(-1, 1).cpu()), 0)     #得到实际标签值
    return total_labels.numpy().flatten(),total_preds.numpy().flatten()               #返回实际标签值和预测值展平后的numpy数据
'''
这里定义了两个函数一个用于数据训练一个用于数据测试，数据训练是通过损失函数再反向传播更新参数，在批量数据中进行训练，下面加入迭代一次一次训练得到更低的损失值
测试函数这里只进行前向传播，不进行反向传播和梯度更新，只进行前向传播和保存预测值和真实值
'''

datasets = [['davis','kiba'][int(sys.argv[1])]]                                       #定义数据集变量，具体哪个数据集由外部参数sys.argv[1]决定，这里可以去搜索下这个外部参数用法理解一下
modeling = [GINConvNet, GATNet, GAT_GCN, GCNNet][int(sys.argv[2])]                    #同样定义模型modeling变量，具体哪一个由外部参数sys.argv[2]决定
model_st = modeling.__name__                                                          #定义模型名字，通过modeling内部属性获取模型名字

cuda_name = "cuda:0"                                                                  #定义cuda_name变量为cuda:0
if len(sys.argv)>3:                                                                   #如果外部参数大于3
    cuda_name = "cuda:" + str(int(sys.argv[3]))                                       #cuda_name由外部参数决定如果sys.argv[3]是1则定义为cuda_name为cuda:1即表示用GPU1
print('cuda_name:', cuda_name)                                                        #打印出最终的cuda_name

TRAIN_BATCH_SIZE = 512                                                                # 定义每批次量数据个数 
TEST_BATCH_SIZE = 512
LR = 0.0005                                                                           #学习率为0.0005
LOG_INTERVAL = 20                                                                     #每多少批次batch，print一下
NUM_EPOCHS = 1000                                                                     #迭代次数

print('Learning rate: ', LR)
print('Epochs: ', NUM_EPOCHS)


#主要项目，在不同数据集中进行迭代
# Main program: iterate over different datasets
for dataset in datasets:                                                              #遍历整个数据集列表
    print('\nrunning on ', model_st + '_' + dataset )                                 #打印出正在用哪个模型训练哪个数据集
    processed_data_file_train = 'data/processed/' + dataset + '_train.pt'             #导入处理好的训练数据文件
    processed_data_file_test = 'data/processed/' + dataset + '_test.pt'
    if ((not os.path.isfile(processed_data_file_train)) or (not os.path.isfile(processed_data_file_test))): #如果这两个数据文件中的任何一个数据文件路径不存在则print出需要通过create_data.py创建数据
        print('please run create_data.py to prepare data in pytorch format!')
    else:
        #导入数据
        train_data = TestbedDataset(root='data', dataset=dataset+'_train')            #满足两个数据集文件都存在的话就用TestbedDataset函数将数据提取出来，这个函数的定义在utils文件中
        test_data = TestbedDataset(root='data', dataset=dataset+'_test')              
        
        #数据批量化
        train_loader = DataLoader(train_data, batch_size=TRAIN_BATCH_SIZE, shuffle=True) #将数据集利用DataLoader划分批次，每一个批次样本数为TR，shuffle=True表示打乱数据
        test_loader = DataLoader(test_data, batch_size=TEST_BATCH_SIZE, shuffle=False)

        #训练模型
        device = torch.device(cuda_name if torch.cuda.is_available() else "cpu")       #定义GPU设备如果cuda_name所指的GPU可用，就用，否则就用cpu
        model = modeling().to(device)                                                  #将模型导入到上边所指定的设备中
        loss_fn = nn.MSELoss()                                                         #定义损失函数为MSE
        optimizer = torch.optim.Adam(model.parameters(), lr=LR)                        #定义优化器为Adam，model.parameters()表示在优化中更新参数，学习率为LR
        best_mse = 1000                                                                #定义最好的mse为1000下边会通过这个去不断迭代
        best_ci = 0                                                                    #最好的ci指数为0
        best_epoch = -1                                                                #最好的迭代次数为-1
        model_file_name = 'model_' + model_st + '_' + dataset +  '.model'              #定义模型文件路径：model_模型名字_数据集.model
        result_file_name = 'result_' + model_st + '_' + dataset +  '.csv'              #定义结果文件：result_模型名字_数据集.csv
        for epoch in range(NUM_EPOCHS):                                                #在定义的NUM_EPOCHS中进行迭代这里是在(0,999)中迭代
            train(model, device, train_loader, optimizer, epoch+1)                     #利用之前定义的训练函数对训练数据集进行训练，参数均利用上边定义的
            G,P = predicting(model, device, test_loader)                               #预测出测试集的labels，得到真实和预测的值，便于下一步计算评估指标
            ret = [rmse(G,P),mse(G,P),pearson(G,P),spearman(G,P),ci(G,P)]              #计算出rmse，mes，pearson，spearman，ci指数这五个指数被保存到csv文件中
            if ret[1]<best_mse:                                                        #如果mse的值小于best_mse
                torch.save(model.state_dict(), model_file_name)                        #则将这个模型保存到模型文件路径中这个save只保存模型权重参数，不保存模型结构
                with open(result_file_name,'w') as f:                                  #以写入模型打开结果文件并定义为f
                    f.write(','.join(map(str,ret)))                                    #将五个评估指标映射为字符串以,隔开后写入到结果文件中
                best_epoch = epoch+1                                                   #更新迭代次数变量 
                best_mse = ret[1]                                                      #更新最好的mse值 
                best_ci = ret[-1]                                                      #更新最好的ci值
                print('rmse improved at epoch ', best_epoch, '; best_mse,best_ci:', best_mse,best_ci,model_st,dataset)    #打印出rmse在这一代中得到提升，最好的迭代次数为：，最好的mse和ci值，模型名称和数据集
            else:
                print(ret[1],'No improvement since epoch ', best_epoch, '; best_mse,best_ci:', best_mse,best_ci,model_st,dataset)   #如果mse没有比上一次好，则打印出没有提升最好的为...